**Tensorflow Version == 2.17
** Python Version == 3.10.12

In [ ]:
import os
import numpy as np
import pandas as pd
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.regularizers import l2
from PIL import UnidentifiedImageError
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0, Xception, MobileNetV2, NASNetMobile
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/Agriculture Classifier/Data"
dir_path = "/content/drive/MyDrive/Agriculture Classifier"

In [ ]:
train_dir = os.path.join(dir_path, 'train')
val_dir = os.path.join(dir_path, 'val')

# Create directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
# Function to check images in all subdirectories
def check_images(directory):
    invalid_images = []  # List to store paths of invalid images

    for root, dirs, files in os.walk(directory):
        for file in files:
            # Only process image files (add more extensions if needed)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img_path = os.path.join(root, file)
                try:
                    img = Image.open(img_path)
                    img.verify()  # Check if the image can be opened
                except (IOError, SyntaxError) as e:
                    invalid_images.append(img_path)  # Store invalid image paths

    # Print summary
    if invalid_images:
        print("The following images are corrupted or in an invalid format:")
        for img_path in invalid_images:
            print(f"- {img_path}")
    else:
        print("All images are in the correct format.")

In [ ]:
# Check images in all subdirectories
check_images(data_path)

All images are in the correct format.


In [ ]:
# Paths
data_path = "/content/drive/MyDrive/Agriculture Classifier/Data"
train_path = "/content/drive/MyDrive/Agriculture Classifier/train"
val_path = "/content/drive/MyDrive/Agriculture Classifier/val"

# Define the train/validation split ratio (e.g., 80% train, 20% validation)
split_ratio = 0.2

# Ensure train and val directories exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# Function to split data into train and validation sets
def split_data():
    for category in os.listdir(data_path):
        category_path = os.path.join(data_path, category)

        # Debugging: Print category path and files
        print(f"Processing category: {category}, Path: {category_path}")

        if os.path.isdir(category_path):
            # Get list of all images in the category folder (filter valid image files)
            images = [f for f in os.listdir(category_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            # Debugging: Print found images
            print(f"Found {len(images)} images in {category}")

            # Skip the category if there are no valid images
            if len(images) == 0:
                print(f"Skipping category {category}: no images found.")
                continue

            # Split the images into training and validation sets
            train_images, val_images = train_test_split(images, test_size=split_ratio, random_state=42)

            # Create corresponding train and val directories for the category
            category_train_path = os.path.join(train_path, category)
            category_val_path = os.path.join(val_path, category)
            os.makedirs(category_train_path, exist_ok=True)
            os.makedirs(category_val_path, exist_ok=True)

            # Move train images
            for image in train_images:
                src_image_path = os.path.join(category_path, image)
                dest_image_path = os.path.join(category_train_path, image)
                shutil.move(src_image_path, dest_image_path)  # Move the image to the train folder

            # Move validation images
            for image in val_images:
                src_image_path = os.path.join(category_path, image)
                dest_image_path = os.path.join(category_val_path, image)
                shutil.move(src_image_path, dest_image_path)  # Move the image to the validation folder

            print(f"Processed category: {category}, Train: {len(train_images)}, Validation: {len(val_images)}")

# Run the data split
split_data()

In [ ]:
# Function to display 2 images from each category
def display_images(folder_path, title):
    # Iterate through each category folder
    for category in os.listdir(folder_path):
        category_path = os.path.join(folder_path, category)

        # Check if it's a directory
        if os.path.isdir(category_path):
            # Get all image files in the category folder
            images = [img for img in os.listdir(category_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

            # Select 2 images (or fewer if less than 2 images exist)
            selected_images = images[:2]

            # Plot the images
            fig, axs = plt.subplots(1, len(selected_images), figsize=(10, 5))
            fig.suptitle(f'{title} - Category: {category}', fontsize=16)

            for i, img_file in enumerate(selected_images):
                img_path = os.path.join(category_path, img_file)
                img = mpimg.imread(img_path)
                axs[i].imshow(img)
                axs[i].axis('off')
                axs[i].set_title(f"{img_file}")

            plt.show()

# Display 2 images from each category in the train and val folders
print("Displaying images from TRAIN folder:")
display_images(train_path, "Train")

print("Displaying images from VAL folder:")
display_images(val_path, "Validation")

In [ ]:
# Load the images from the training directory
train_dir = "/content/drive/MyDrive/Agriculture Classifier/train"
val_dir = "/content/drive/MyDrive/Agriculture Classifier/val"

# Image parameters
img_height, img_width = 224, 224  # VGG16 input size
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values
    rotation_range=40,         # Randomly rotate images
    width_shift_range=0.2,     # Randomly shift the width
    height_shift_range=0.2,    # Randomly shift the height
    shear_range=0.2,           # Shear transformation
    zoom_range=0.2,            # Zoom in/out
    horizontal_flip=True,      # Randomly flip images
    fill_mode='nearest'        # Fill in pixels after transformations
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale validation data

# Load train and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Image parameters
img_height, img_width = 224, 224  # VGG16 input size
batch_size = 32

# Functional API - Define the input layer
input_tensor = Input(shape=(img_height, img_width, 3))  # Explicit input shape

# Load VGG16 pre-trained model, excluding the top layer (for classification)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the VGG16 layers to avoid retraining them
for layer in vgg16_base.layers:
    layer.trainable = False

# Add custom layers on top of VGG16 base
x = vgg16_base.output  # Output from the base model
x = Flatten()(x)  # Flatten the output
x = Dense(256, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
output_tensor = Dense(30, activation='softmax')(x)  # Output layer for 30 categories

# Create the model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
optimizer = Adam(learning_rate=1e-4)  # Adam optimizer with a small learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Callbacks for learning rate reduction and early stopping
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.0412 - loss: 3.6491 - val_accuracy: 0.0904 - val_loss: 3.2509 - learning_rate: 1.0000e-04
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 560ms/step - accuracy: 0.0948 - loss: 3.2844 - val_accuracy: 0.1695 - val_loss: 3.0871 - learning_rate: 1.0000e-04
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 542ms/step - accuracy: 0.1182 - loss: 3.1488 - val_accuracy: 0.1864 - val_loss: 2.9631 - learning_rate: 1.0000e-04
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 549ms/step - accuracy: 0.1540 - loss: 3.0459 - val_accuracy: 0.2147 - val_loss: 2.8608 - learning_rate: 1.0000e-04
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 469ms/step - accuracy: 0.2038 - loss: 2.8871 - val_accuracy: 0.2542 - val_loss: 2.7325 - learning_rate: 1.0000e-04
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 483ms/step - accuracy: 0.2238 - loss: 2.7768 - val_accuracy: 0.2825 - val_loss: 2.6980 - learning_rate: 1.0000e-04
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 573ms/step - accuracy: 0.2529 

In [ ]:
# Load VGG19 pre-trained model, excluding the top layer (for classification)
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the VGG19 layers to avoid retraining them
for layer in vgg19_base.layers:
    layer.trainable = False

# Add custom layers on top of VGG19 base
x = vgg19_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model2 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model2.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model2.summary()

In [25]:
# Train the model
history = model2.fit(
    train_generator,
    epochs=30,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on validation set
val_loss, val_acc = model2.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 654ms/step - accuracy: 0.0509 - loss: 3.7405 - val_accuracy: 0.0847 - val_loss: 3.2843 - learning_rate: 1.0000e-04
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 549ms/step - accuracy: 0.0822 - loss: 3.3655 - val_accuracy: 0.1017 - val_loss: 3.1954 - learning_rate: 1.0000e-04
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 536ms/step - accuracy: 0.0962 - loss: 3.2313 - val_accuracy: 0.2034 - val_loss: 3.0645 - learning_rate: 1.0000e-04
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 619ms/step - accuracy: 0.1114 - loss: 3.1576 - val_accuracy: 0.1751 - val_loss: 3.0084 - learning_rate: 1.0000e-04
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 620ms/step - accuracy: 0.1410 - loss: 3.0674 - val_accuracy: 0.2147 - val_loss: 2.9227 - learning_rate: 1.0000e-04
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 703ms/step - accuracy: 0.1535 - loss: 3.0310 - val_accuracy: 0.2712 - val_loss: 2.8293 - learning_rate: 1.0000e-04
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 570ms/step - acc

In [ ]:
# Load ResNet50 pre-trained model, excluding the top layer (for classification)
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the ResNet50 layers to avoid retraining them
for layer in resnet50_base.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50 base
x = resnet50_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model3 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model3.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model3.summary()

In [27]:
# Train the model
history = model3.fit(
    train_generator,
    epochs=30,  # Adjust the number of epochs as needed
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on validation set
val_loss, val_acc = model3.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0364 - loss: 3.9565 - val_accuracy: 0.0282 - val_loss: 3.4120 - learning_rate: 1.0000e-04
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 586ms/step - accuracy: 0.0289 - loss: 3.4085 - val_accuracy: 0.0339 - val_loss: 3.4036 - learning_rate: 1.0000e-04
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 578ms/step - accuracy: 0.0476 - loss: 3.3998 - val_accuracy: 0.0452 - val_loss: 3.3998 - learning_rate: 1.0000e-04
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 588ms/step - accuracy: 0.0483 - loss: 3.4011 - val_accuracy: 0.0452 - val_loss: 3.4008 - learning_rate: 1.0000e-04
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 424ms/step - accuracy: 0.0533 - loss: 3.4018 - val_accuracy: 0.0452 - val_loss: 3.4006 - learning_rate: 1.0000e-04
Epoch 6/30
20/21 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step - accuracy: 0.0444 - loss: 3.4010
Epoch 6: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 479ms/step - accuracy: 0

In [ ]:
# Load EfficientNetB0 pre-trained model, excluding the top layer (for classification)
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the EfficientNetB0 layers to avoid retraining them
for layer in efficientnet_base.layers:
    layer.trainable = False

# Add custom layers on top of EfficientNetB0 base
x = efficientnet_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model4 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model4.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model4.summary()

In [29]:
# Train the model
history = model4.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
val_loss, val_acc = model4.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.0416 - loss: 4.0546 - val_accuracy: 0.0395 - val_loss: 3.4017 - learning_rate: 1.0000e-04
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 485ms/step - accuracy: 0.0223 - loss: 3.4042 - val_accuracy: 0.0452 - val_loss: 3.4011 - learning_rate: 1.0000e-04
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 545ms/step - accuracy: 0.0307 - loss: 3.4073 - val_accuracy: 0.0452 - val_loss: 3.4011 - learning_rate: 1.0000e-04
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 544ms/step - accuracy: 0.0460 - loss: 3.4012 - val_accuracy: 0.0282 - val_loss: 3.4139 - learning_rate: 1.0000e-04
Epoch 5/20
20/21 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.0329 - loss: 3.4096
Epoch 5: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
21/21 ━━━━━━━━━━━━━━━━━━━━ 18s 486ms/step - accuracy: 0.0337 - loss: 3.4094 - val_accuracy: 0.0395 - val_loss: 3.4015 - learning_rate: 1.0000e-04
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 526ms/step - accuracy: 0

In [ ]:
# Load MobileNetV2 pre-trained model, excluding the top layer (for classification)
mobilenetv2_base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the MobileNetV2 layers to avoid retraining them
for layer in mobilenetv2_base.layers:
    layer.trainable = False

# Add custom layers on top of MobileNetV2 base
x = mobilenetv2_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model5 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model5.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model5.summary()

In [31]:
# Train the model
history = model5.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
val_loss, val_acc = model5.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 29s 814ms/step - accuracy: 0.0864 - loss: 4.7396 - val_accuracy: 0.2203 - val_loss: 2.7934 - learning_rate: 1.0000e-04
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 30s 459ms/step - accuracy: 0.1613 - loss: 3.0188 - val_accuracy: 0.3446 - val_loss: 2.3819 - learning_rate: 1.0000e-04
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 410ms/step - accuracy: 0.2104 - loss: 2.7291 - val_accuracy: 0.4407 - val_loss: 2.1064 - learning_rate: 1.0000e-04
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 433ms/step - accuracy: 0.3415 - loss: 2.3665 - val_accuracy: 0.4689 - val_loss: 1.8074 - learning_rate: 1.0000e-04
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 486ms/step - accuracy: 0.4070 - loss: 2.2103 - val_accuracy: 0.5706 - val_loss: 1.5777 - learning_rate: 1.0000e-04
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 481ms/step - accuracy: 0.4563 - loss: 1.9156 - val_accuracy: 0.6215 - val_loss: 1.5296 - learning_rate: 1.0000e-04
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 416ms/step - acc

In [ ]:
# Load NASNetMobile pre-trained model, excluding the top layer (for classification)
nasnet_base = NASNetMobile(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the NASNetMobile layers to avoid retraining them
for layer in nasnet_base.layers:
    layer.trainable = False

# Add custom layers on top of NASNetMobile base
x = nasnet_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model6 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model6.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model6.summary()

In [33]:
# Train the model
history = model6.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
val_loss, val_acc = model6.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.0626 - loss: 4.3999 - val_accuracy: 0.2373 - val_loss: 2.8500 - learning_rate: 1.0000e-04
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 513ms/step - accuracy: 0.1718 - loss: 2.9885 - val_accuracy: 0.3333 - val_loss: 2.5023 - learning_rate: 1.0000e-04
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 424ms/step - accuracy: 0.2675 - loss: 2.6486 - val_accuracy: 0.4294 - val_loss: 2.2107 - learning_rate: 1.0000e-04
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 490ms/step - accuracy: 0.3443 - loss: 2.3573 - val_accuracy: 0.4520 - val_loss: 2.0880 - learning_rate: 1.0000e-04
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 466ms/step - accuracy: 0.3921 - loss: 2.3015 - val_accuracy: 0.4520 - val_loss: 1.9472 - learning_rate: 1.0000e-04
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 518ms/step - accuracy: 0.4236 - loss: 2.1340 - val_accuracy: 0.4859 - val_loss: 1.8377 - learning_rate: 1.0000e-04
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 514ms/step - accura

In [ ]:
# Load Xception pre-trained model, excluding the top layer (for classification)
xception_base = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the Xception layers to avoid retraining them
for layer in xception_base.layers:
    layer.trainable = False

# Add custom layers on top of Xception base
x = xception_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(30, activation='softmax')(x)

# Create the model
model7 = Model(inputs=input_tensor, outputs=output_tensor)

# Compile and train
model7.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model7.summary()

In [35]:
# Train the model
history = model7.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
val_loss, val_acc = model7.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1063 - loss: 3.6416 - val_accuracy: 0.4068 - val_loss: 2.3656 - learning_rate: 1.0000e-04
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 505ms/step - accuracy: 0.2530 - loss: 2.6715 - val_accuracy: 0.4633 - val_loss: 1.8794 - learning_rate: 1.0000e-04
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 500ms/step - accuracy: 0.3739 - loss: 2.2552 - val_accuracy: 0.5141 - val_loss: 1.7458 - learning_rate: 1.0000e-04
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 16s 554ms/step - accuracy: 0.4722 - loss: 1.9466 - val_accuracy: 0.5706 - val_loss: 1.5841 - learning_rate: 1.0000e-04
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 521ms/step - accuracy: 0.4916 - loss: 1.8185 - val_accuracy: 0.5367 - val_loss: 1.5951 - learning_rate: 1.0000e-04
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 501ms/step - accuracy: 0.4631 - loss: 1.8003 - val_accuracy: 0.6158 - val_loss: 1.4932 - learning_rate: 1.0000e-04
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 20s 509ms/step - accura

In [37]:
# Define the directory path
model_dir = '/content/drive/MyDrive/Agriculture Classifier/Model'

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Save the model in the "Model" folder
model5.save(os.path.join(model_dir, 'MobileNetV2.h5'))

print(f"Model saved at {os.path.join(model_dir, 'MobileNetV2.h5')}")

Model saved at /content/drive/MyDrive/Agriculture Classifier/Model/MobileNetV2.h5
